# 03 Vector DB

## Intro 

Azure Cognitive Search can serve as a vector database by storing embeddings, which represent semantic information, in its index. This enables efficient querying and retrieval of documents or data points based on their semantic content, making it suitable for a wide range of applications, including search engines.

## Step 1 - Read Environment / Create client (OpenAI / Search) instances

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.8"
#r "nuget: Azure.Search.Documents, 11.5.0-beta.4"
#r "nuget: DotNetEnv, 2.5.0"

using Azure;
using Azure.AI.OpenAI;
using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Azure.Search.Documents.Models;
using DotNetEnv;


string _configurationFile = @"../01/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY") ?? "";
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") ?? "";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "";

string cognitiveSearchEndpoint = Environment.GetEnvironmentVariable("COGNITIVESEARCH_ENDPOINT") ?? "";
string cognitiveSearchApiKey = Environment.GetEnvironmentVariable("COGNITIVESEARCH_APIKEY") ?? "";

Console.WriteLine("Environment variables loaded...");  

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

AzureKeyCredential searchCredential = new AzureKeyCredential(cognitiveSearchApiKey);
SearchIndexClient searchIndexClient = new SearchIndexClient(new Uri(cognitiveSearchEndpoint), searchCredential);

Console.WriteLine("Azure OpenAI client created...");
Console.WriteLine("Cognitive Search client created...");    


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.8 Azure.Search.Documents, 11.5.0-beta.4 DotNetEnv, 2.5.0

Environment variables loaded...
Azure OpenAI client created...
Cognitive Search client created...


## Step 2 - Create Search Index

In [2]:
//Create search index
string indexName = "fact-index"; 
string searchConfigName = "fact-config";

int modelDimensions = 1536;
SearchIndex searchIndex = new(indexName)
{
    Fields =
    {
        new SimpleField("FactId", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true, IsFacetable = true },
        new SearchableField("FactName") { IsFilterable = true, IsSortable = true },
        new SearchableField("FactDescription") { IsFilterable = true },
        new SearchField("FactVector", SearchFieldDataType.Collection(SearchFieldDataType.Single))
        {
            IsSearchable = true,
            VectorSearchDimensions = modelDimensions,
            VectorSearchConfiguration = searchConfigName
        },
    },
    VectorSearch = new()
    {
        AlgorithmConfigurations =
        {
            new HnswVectorSearchAlgorithmConfiguration(searchConfigName)
        }
    }
}; 

try { await searchIndexClient.DeleteIndexAsync(indexName); } catch {}
await searchIndexClient.CreateIndexAsync(searchIndex);   
Console.WriteLine("Search index created...");


Search index created...


## Step 3 - Define facts

In [3]:
//Define facts
public class Fact
{
    public string FactId { get; set; } = "";
    public string FactName { get; set; } = "";
    public string FactDescription { get; set; } = "";
    public IReadOnlyList<float> FactVector { get; set; } = new List<float>(); 
}

Fact[] facts = new[]
{
    new Fact()
    {
        FactId = "1",
        FactName = "Company Music",
        FactDescription = @"Firma Musik is one of the world's leading record labels. 
                            It has signed famous singers and is very profitable! 
                            The flagship of Contoso Music is a group that performs under the name 'Contoso Only'!",
        FactVector = new float[1536],
    },
    new Fact()
    {
        FactId = "2",
        FactName = "Company Maritim",
        FactDescription = @"Company Heavy Industry Maritime products. 
                        The current bestseller is the transporter 'Contoso XL Heavy 2000'.",
        FactVector = new float[1536],
    },
    new Fact()
    {
        FactId = "3",
        FactName = "Company Agriculture",
        FactDescription = @"Company Agriculture is a German start-up that focuses on the production of milk and grain. 
                            Since this is a start-up, no further information is available!",
        FactVector = new float[1536], 
    },
};

Console.WriteLine("Facts defined...");

Facts defined...


## Step 4 - Upload facts to Azure Cognitive Search

In [4]:
foreach(Fact fact in facts) {
    EmbeddingsOptions factEmbeddingsOptions = new EmbeddingsOptions(fact.FactDescription);
    var factEmbedding = await openAIClient.GetEmbeddingsAsync(embeddingDeploymentName, factEmbeddingsOptions);
    float[] embeddingData = factEmbedding.Value.Data[0].Embedding.ToArray<float>();
    fact.FactVector = embeddingData;
}

SearchClient searchClient = searchIndexClient.GetSearchClient(indexName);
await searchClient.IndexDocumentsAsync(IndexDocumentsBatch.Upload(facts));

Console.WriteLine("Fact documents uploaded...");

Fact documents uploaded...


## Step 5 - Vector search

In [7]:
//Search vectors
EmbeddingsOptions embeddingsOptions; 
embeddingsOptions = new EmbeddingsOptions("Who produces Container Ships?");
var embedding = await openAIClient.GetEmbeddingsAsync(embeddingDeploymentName, embeddingsOptions);
float[] vectorizedResult = embedding.Value.Data[0].Embedding.ToArray<float>();


SearchResults<Fact> response = await searchClient.SearchAsync<Fact>(
    null,
    new SearchOptions {
        Vectors = { new() { Value = vectorizedResult, KNearestNeighborsCount = 1, Fields = { "FactVector" } } },
    }
);

Console.WriteLine($"Single Vector Search Results:");
await foreach (SearchResult<Fact> result in response.GetResultsAsync())
{
    Console.WriteLine($"Search result: {result.Document.FactId}: {result.Document.FactName}");
}

Single Vector Search Results:
Search result: 2: Company Maritim
